In [1]:
import pdb
import os
import sys
from ctypes import (
    CDLL,
    POINTER,
    Structure,
    byref,
    string_at,
    c_char_p,
    c_int32,
    c_int64,
    c_uint64,
    c_ubyte,
)

from ctypes.util import find_library
from typing import Optional, Union

LIB: CDLL = None

class FfiByteBuffer(Structure):
    """A byte buffer allocated by python."""
    _fields_ = [
        ("length", c_int64),
        ("data", POINTER(c_ubyte)),
    ]


class FfiError(Structure):
    """An error allocated by python."""
    _fields_ = [
        ("code", c_int32),
        ("message", c_char_p),
    ]


def _decode_bytes(arg: Optional[Union[str, bytes, FfiByteBuffer]]) -> bytes:
    if isinstance(arg, FfiByteBuffer):
        return string_at(arg.data, arg.length)
    if isinstance(arg, memoryview):
        return string_at(arg.obj, arg.nbytes)
    if isinstance(arg, bytearray):
        return arg
    if arg is not None:
        if isinstance(arg, str):
            return arg.encode("utf-8")
    return bytearray()


def _encode_bytes(arg: Optional[Union[str, bytes, FfiByteBuffer]]) -> FfiByteBuffer:
    if isinstance(arg, FfiByteBuffer):
        return arg
    buf = FfiByteBuffer()
    if isinstance(arg, memoryview):
        buf.length = arg.nbytes
        if arg.contiguous and not arg.readonly:
            buf.data = (c_ubyte * buf.length).from_buffer(arg.obj)
        else:
            buf.data = (c_ubyte * buf.length).from_buffer_copy(arg.obj)
    elif isinstance(arg, bytearray):
        buf.length = len(arg)
        if buf.length > 0:
            buf.data = (c_ubyte * buf.length).from_buffer(arg)
    elif arg is not None:
        if isinstance(arg, str):
            arg = arg.encode("utf-8")
        buf.length = len(arg)
        if buf.length > 0:
            buf.data = (c_ubyte * buf.length).from_buffer_copy(arg)
    return buf


def _load_library(lib_name: str) -> CDLL:
    lib_prefix_mapping = {"win32": ""}
    lib_suffix_mapping = {"darwin": ".dylib", "win32": ".dll"}
    try:
        os_name = sys.platform
        lib_prefix = lib_prefix_mapping.get(os_name, "lib")
        lib_suffix = lib_suffix_mapping.get(os_name, ".so")
        lib_path = os.path.join(
            os.path.dirname(os.getcwd()), f"agora-allosaurus-rs/target/release/{lib_prefix}{lib_name}{lib_suffix}"
        )
        return CDLL(lib_path)
    except KeyError:
        print ("Unknown platform for shared library")
    except OSError:
        print ("Library not loaded from python package")

    lib_path = find_library(lib_name)
    if not lib_path:
        if sys.platform == "darwin":
            ld = os.getenv("DYLD_LIBRARY_PATH")
            lib_path = os.path.join(ld, "liboberon.dylib")
            if os.path.exists(lib_path):
                return CDLL(lib_path)

            ld = os.getenv("DYLD_FALLBACK_LIBRARY_PATH")
            lib_path = os.path.join(ld, "liboberon.dylib")
            if os.path.exists(lib_path):
                return CDLL(lib_path)
        elif sys.platform != "win32":
            ld = os.getenv("LD_LIBRARY_PATH")
            lib_path = os.path.join(ld, "liboberon.so")
            if os.path.exists(lib_path):
                return CDLL(lib_path)

        raise Exception(f"Error loading library: {lib_name}")
    try:
        return CDLL(lib_path)
    except OSError as e:
        raise Exception(f"Error loading library: {lib_name}")


def _get_library() -> CDLL:
    global LIB
    if LIB is None:
        LIB = _load_library("agora_allosaurus_rs")

    return LIB

def _get_func(fn_name: str):
    return getattr(_get_library(), fn_name)

def _free_buffer(buffer: FfiByteBuffer):
    lib_fn = _get_func("allosaurus_byte_buffer_free")
    lib_fn(byref(buffer))


def _free_string(err: FfiError):
    lib_fn = _get_func("allosaurus_string_free")
    lib_fn(byref(err))


def _free_handle(handle: c_int64, err: FfiError):
    lib_fn = _get_func("allosaurus_create_proof_free")
    lib_fn(handle, byref(err))

In [2]:
def new_server() -> c_int64:
    err = FfiError()
    lib_fn = _get_func("allosaurus_new_server")
    lib_fn.restype = c_uint64

    handle = lib_fn(byref(err))
    if handle == 0:
        message = string_at(err.message)
        raise Exception(message)
    handle = c_uint64(handle)
    return handle
server = new_server()
server

c_ulong(4707384675830595586)

In [3]:
def new_user(server) -> c_int64:
    buffer = FfiByteBuffer()
    err = FfiError()
    lib_fn = _get_func("allosaurus_new_user")
    lib_fn(server, byref(buffer), byref(err))

    if not buffer:
        message = string_at(err.message)
        raise Exception(message)
    buffer = _decode_bytes(buffer)
    return buffer
user = new_user(server)
user

b'T\xaa|\xac\x07\x1c\xfc\xca\x15\x08AS\xe8\xd1\x96\xc4\xca\x00\xd1%[\x16\x8d&\xb4\xd6\xacNQ\t\xbaq\x00\xb0\xe1\x10\xdc98\xb3\x12rF\xc7L\x94\xff\x8e\x96=L\x03\xe6\xda\xdf\xb4\xe13\xe4\x99d\xbd\xcf\xea\x1ax\xf2^\x88l\xf6\xf3\xca6T\x08N\x11GNK\x93\x9c9F\xee\xd3@\x03\xfb\x8e\xbb\xc1\xfa\xc5^@\xc6\xf0t\x81#Y\x93WC}\xd3\x8a\x91\xdb.\xa3~\xfb\xce\xdf\xd1\x1b\xe6q\x969,&\x8dP\x83\xfe\x0c8{\xbcf\xe5\xf5\x9bbM\xcccc\xccH\xb3\xd9\x02\xaa5\x032\'\xfc\xe5\xabT\x14\xc7\xf3\xc8T\x82_\xfe\xedN\xfa\x08\x9d&\x04\xd1=\x03\xb6\xd7A\xad\x94*To\x87K\x01"\xa3\x01\x9a\xef}\x04\xcf5Z\xc3\xe0yN\xbf\x04K\xc7G\x05\x7f&b\xf1\xb0\xec\xad\xa1\xb9\xbc8b\xdf\xfaP`PFQ\x7f\x16i\xcbi.\xe7\xe9\x97oX\xdf\xc4\xa54\xad\xa6s\x99\x16\x14&#3\xe6\xf7L\x8cA\x99\n5d\x05\xdeg\x7f\xef\xb2\x8a\xf1~\x0cG\x8ca\n\xfd\x16\x01'

In [4]:
def server_add(server, user) -> c_int64:
    buffer = FfiByteBuffer()
    err = FfiError()
    lib_fn = _get_func("allosaurus_server_add")
    lib_fn(server, _encode_bytes(user), byref(buffer), byref(err))
    if not buffer:
        message = string_at(err.message)
        raise Exception(message)
    buffer = _decode_bytes(buffer)
    return buffer
server_add(server, user)

b'\xab\xedc\xd3IvKJ\x04T>z\xbb5<\x14\x1d\xe2\x0b\xccl\x9e\xbe\xb7OA\xc9\xeb\xf9\xb5\xb6\x9e<\xb5\x0c\xd3"\xe7,\xf4\xf7\xa8\xe5\xe9\x00\xe3\xf3\xc9'

In [5]:
def server_delete(server, user) -> c_int64:
    buffer = FfiByteBuffer()
    err = FfiError()
    lib_fn = _get_func("allosaurus_server_delete")
    lib_fn(server, _encode_bytes(user), byref(buffer), byref(err))
    if not buffer:
        message = string_at(err.message)
        raise Exception(message)
    buffer = _decode_bytes(buffer)
    return buffer
# server_delete(server, user)

In [6]:
def server_get_epoch(server) -> int:
    err = FfiError()
    lib_fn = _get_func("allosaurus_server_get_epoch")
    return lib_fn(server, byref(err))
server_get_epoch(server)

1

In [7]:
def server_get_accumulator(server) -> c_int64:
    buffer = FfiByteBuffer()
    err = FfiError()
    lib_fn = _get_func("allosaurus_server_get_accumulator")
    lib_fn(server, byref(buffer), byref(err))
    if not buffer:
        message = string_at(err.message)
        raise Exception(message)
    buffer = _decode_bytes(buffer)
    return buffer
server_get_accumulator(server)

b'\xb0\xe1\x10\xdc98\xb3\x12rF\xc7L\x94\xff\x8e\x96=L\x03\xe6\xda\xdf\xb4\xe13\xe4\x99d\xbd\xcf\xea\x1ax\xf2^\x88l\xf6\xf3\xca6T\x08N\x11GNK'

In [8]:
def server_get_witness_public_key(server) -> c_int64:
    buffer = FfiByteBuffer()
    err = FfiError()
    lib_fn = _get_func("allosaurus_server_get_witness_public_key")
    lib_fn(server, byref(buffer), byref(err))
    if not buffer:
        message = string_at(err.message)
        raise Exception(message)
    buffer = _decode_bytes(buffer)
    return buffer
server_get_witness_public_key(server)

b"\x93\x9c9F\xee\xd3@\x03\xfb\x8e\xbb\xc1\xfa\xc5^@\xc6\xf0t\x81#Y\x93WC}\xd3\x8a\x91\xdb.\xa3~\xfb\xce\xdf\xd1\x1b\xe6q\x969,&\x8dP\x83\xfe\x0c8{\xbcf\xe5\xf5\x9bbM\xcccc\xccH\xb3\xd9\x02\xaa5\x032'\xfc\xe5\xabT\x14\xc7\xf3\xc8T\x82_\xfe\xedN\xfa\x08\x9d&\x04\xd1=\x03\xb6\xd7A"

In [9]:
def server_get_sign_public_key(server) -> c_int64:
    buffer = FfiByteBuffer()
    err = FfiError()
    lib_fn = _get_func("allosaurus_server_get_sign_public_key")
    lib_fn(server, byref(buffer), byref(err))
    if not buffer:
        message = string_at(err.message)
        raise Exception(message)
    buffer = _decode_bytes(buffer)
    return buffer
server_get_sign_public_key(server)

b'\xad\x94*To\x87K\x01"\xa3\x01\x9a\xef}\x04\xcf5Z\xc3\xe0yN\xbf\x04K\xc7G\x05\x7f&b\xf1\xb0\xec\xad\xa1\xb9\xbc8b\xdf\xfaP`PFQ\x7f\x16i\xcbi.\xe7\xe9\x97oX\xdf\xc4\xa54\xad\xa6s\x99\x16\x14&#3\xe6\xf7L\x8cA\x99\n5d\x05\xdeg\x7f\xef\xb2\x8a\xf1~\x0cG\x8ca\n\xfd\x16'

In [10]:
def server_get_public_keys(server) -> c_int64:
    buffer = FfiByteBuffer()
    err = FfiError()
    lib_fn = _get_func("allosaurus_server_get_public_keys")
    lib_fn(server, byref(buffer), byref(err))
    if not buffer:
        message = string_at(err.message)
        raise Exception(message)
    buffer = _decode_bytes(buffer)
    return buffer
server_get_public_keys(server)

b'\x93\x9c9F\xee\xd3@\x03\xfb\x8e\xbb\xc1\xfa\xc5^@\xc6\xf0t\x81#Y\x93WC}\xd3\x8a\x91\xdb.\xa3~\xfb\xce\xdf\xd1\x1b\xe6q\x969,&\x8dP\x83\xfe\x0c8{\xbcf\xe5\xf5\x9bbM\xcccc\xccH\xb3\xd9\x02\xaa5\x032\'\xfc\xe5\xabT\x14\xc7\xf3\xc8T\x82_\xfe\xedN\xfa\x08\x9d&\x04\xd1=\x03\xb6\xd7A\xad\x94*To\x87K\x01"\xa3\x01\x9a\xef}\x04\xcf5Z\xc3\xe0yN\xbf\x04K\xc7G\x05\x7f&b\xf1\xb0\xec\xad\xa1\xb9\xbc8b\xdf\xfaP`PFQ\x7f\x16i\xcbi.\xe7\xe9\x97oX\xdf\xc4\xa54\xad\xa6s\x99\x16\x14&#3\xe6\xf7L\x8cA\x99\n5d\x05\xdeg\x7f\xef\xb2\x8a\xf1~\x0cG\x8ca\n\xfd\x16'

In [11]:
def user_create_witness(server, user):
    err = FfiError()
    lib_fn = _get_func("allosaurus_user_create_witness")
    lib_fn(server, _encode_bytes(user), byref(err))
    if err.code != 0:
        message = string_at(err.message)
        raise Exception(message)
    return "Witness created successfully"
user_create_witness(server, user)

'Witness created successfully'

Witness created for user Element(Scalar(0x54aa7cac071cfcca15084153e8d196c4ca00d1255b168d26b4d6ac4e5109ba71))


In [12]:
def user_make_membership_proof(server, user) -> c_int64:
    buffer = FfiByteBuffer()
    err = FfiError()
    lib_fn = _get_func("allosaurus_user_make_membership_proof")
    lib_fn(server, _encode_bytes(user), byref(buffer), byref(err))
    if not buffer:
        message = string_at(err.message)
        raise Exception(message)
    buffer = _decode_bytes(buffer)
    return buffer
proof = user_make_membership_proof(server, user)
proof

Witness created for user Element(Scalar(0x54aa7cac071cfcca15084153e8d196c4ca00d1255b168d26b4d6ac4e5109ba71))


b'\x92\x88pm\x01.\x9f\xf0\x1d!\x91\xa4CU*\x06\r\x17j\x80\xec\x8a\x8f1\t\xa5\x07\xb0I\xbd\x8c.49`&6\x17\xae_\xbb\xc9\xdb\x83$\xc8\xc4H\x80\x86\xacR|e\xb7\xf0\xd3x\xee\xdf/\xf6A/$\x88\x14hM\r\x89~|\r\xbe4\xa9\x01U^\xb5\x8a\xea\xcd\xf6\x08\xaf\xd3D\xa7\xe4W\x87\x8e4\xc8\xa6\xd3&\xe1J!\xa9\xb1[MV<%\xc7\xa6\xfd\xdc\x85yL\xf5\xd3\xeei\xb1F\x11\xfb\x19\xd6\xcd\xc8\xf0N>\x1a\r\xac0\'\xbcLQ\x10\xec)\xe3\x03r=\xc7\x11\x03\xe7j\x0b\xf4\x99\xb9\xb1f\x96yD\xde\xc9\x0c\xde\x17\xa5\x84!\x0bbC\xf4k\xa9@\xab\x157\x9c\xf8A\x9c/\x9e\xc6\xc6\xfew"\x84\xab]i\xa5\xdb\xc4\x8al\x1b\r\xad\x02C\xcdCF\x0b@j*+&T\x81\x9b4]\xceeO%#!f\xf1H\xc4M\xaf\xa32E\xbf\xa7dmS\xbd\x96\xfe\x1c:B\x7f4K\xc2f\x1a\xfae\xcaY\xbaQ\xd2ci\x8c\x0e\xb2l\xaa\x1a\x1a\xc5\xa4UU\xa1\x1a\xe6lJ\xa5nQ*\xd6\x96Q\xda\xfc\xcc\xf2\xd76\xbc$>#;\xab\x84\x93\x87\xff\xb2n\x8f&\xa5N\x89\x13\xf1@\xdb\x12\xea\x89\x8e\xc6\xf7\xae\xc4,\x8c\xf7X\x9c\n\x9d+\x8e\xb5[@\xb4\x14\xe9E)mG\xbf!\xffBN\xd00\x07\xa7\xa8\xeb\x17u\xf6)\xd3\xcc\xbfI9\xb0\x81]M=\xde\xc7\xba

In [13]:
def witness_check_membership_proof(server, proof) -> c_int64:
    err = FfiError()
    lib_fn = _get_func("allosaurus_witness_check_membership_proof")
    lib_fn(server, _encode_bytes(proof), byref(err))
    if err.code != 0:
        message = string_at(err.message)
        raise Exception(message)
    return "Membership proof verified successfully"
witness_check_membership_proof(server, proof)

'Membership proof verified successfully'

In [14]:
def check_witness(server, user):
    err = FfiError()
    lib_fn = _get_func("allosaurus_user_check_witness")
    lib_fn(server, _encode_bytes(user), byref(err))
    if err.code != 0:
        message = string_at(err.message)
        raise Exception(message)
    return "Witness is valid"
check_witness(server, user)

Witness created for user Element(Scalar(0x54aa7cac071cfcca15084153e8d196c4ca00d1255b168d26b4d6ac4e5109ba71))
Checking witness for user Element(Scalar(0x54aa7cac071cfcca15084153e8d196c4ca00d1255b168d26b4d6ac4e5109ba71))


'Witness is valid'

In [15]:
def server_batch_delete(server, user_list):
    buffer = FfiByteBuffer()
    err = FfiError()
    lib_fn = _get_func("allosaurus_server_update")
    lib_fn(*user_list, len(user_list), server, byref(buffer), byref(err))
    if not buffer:
        message = string_at(err.message)
        raise Exception(message)
    buffer = _decode_bytes(buffer)
    return buffer


In [16]:
user1 = new_user(server)
user2 = new_user(server)
user3 = new_user(server)
server_add(server, user1)


b'\xb8 \xa9D\x18\x819\xc5\x8b7\xbd\x1d\x87N\x8b\x97B1\xc0d\xfb\xeb47\xf4DSzI7E*@\xb0\xe0\xf6\xf8\xd1\x9a;D\x1e\xe7\xf0\xb1{a\x12'

In [ ]:
server_batch_delete(server, [user1, user2, user3])

: 